# Tutorial 2: Spyker and Numpy
This tutorial follows the previous tutoral on how to use the library and its interaction with PyTorch. In this tutorial we will see how to use the library with Numpy to classify the MNIST dataset. First We start with importing the needed tools.

In [ ]:
import spyker
import numpy as np
from sklearn.svm import SVC
from sklearn.decomposition import PCA

We need to load the data as in batches to speed up the process. So we define the `Dataset` class that iterates over the dataset batch by batch.

In [ ]:
class Dataset:
    def __init__(self, data, target, batch):
        self.data = data
        self.target = target
        self.batch = batch

    def __len__(self):
        return (self.data.shape[0] + self.batch - 1) // self.batch

    def __getitem__(self, index):
        if index >= len(self): raise StopIteration
        start = index * self.batch
        end = min(start + self.batch, self.data.shape[0])
        return spyker.to_tensor(self.data[start:end]), self.target[start:end]

Then we can wrap the training and testing sets with our `Dataset` class. 

In [ ]:
batch, root = 64, './MNIST'
trainx, trainy, testx, testy = spyker.read_mnist(
    root+'/train-images-idx3-ubyte', root+'/train-labels-idx1-ubyte',
    root+'/t10k-images-idx3-ubyte', root+'/t10k-labels-idx1-ubyte')
trainx, trainy, testx, testy = spyker.to_numpy(trainx, trainy, testx, testy)
train = Dataset(trainx, trainy, batch)
test = Dataset(testx, testy, batch)

The `Transform` and `Network` classes are the same as before.

In [ ]:
class Transform:
    def __init__(self, device):
        self.device = device
        self.filter = spyker.LoG(3, [.5, 1, 2], pad=3, device=device)
    
    def __call__(self, input):
        input = input.to(self.device)
        return spyker.code(spyker.threshold(self.filter(input), .01), 15)

In [ ]:
class Network:
    def __init__(self, device):
        self.conv1 = spyker.Conv(6, 50, 5, pad=2, device=device)
        self.conv2 = spyker.Conv(50, 100, 3, pad=1, device=device)
        self.conv1.stdpconfig = [spyker.STDPConfig(.004, -.003)]
        self.conv2.stdpconfig = [spyker.STDPConfig(.004, -.003)]
    
    def train1(self, input):
        output = spyker.inhibit(spyker.threshold(self.conv1(input), 16))
        self.conv1.stdp(input, spyker.convwta(output, 3, 5), spyker.fire(output))
        
    def train2(self, input):
        input = spyker.pool(spyker.fire(self.conv1(input), 16), 2)
        output = spyker.inhibit(spyker.threshold(self.conv2(input), 5))
        self.conv2.stdp(input, spyker.convwta(output, 1, 8), spyker.fire(output))
    
    def __call__(self, input):
        input = spyker.pool(spyker.fire(self.conv1(input), 16), 2)
        input = spyker.pool(spyker.fire(self.conv2(input), 5), 3)
        input = 15 - spyker.to_numpy(spyker.gather(input))
        return input.reshape(input.shape[0], -1)

We will use `Numpy` to concatenate the output tensors in the `Total` function.

In [ ]:
def Total(network, transform, dataset):
    data_total, target_total = [], []
    for data, target in dataset:
        data_total.append(network(transform(data)))
        target_total.append(target)
    return np.concatenate(data_total), np.concatenate(target_total)

The rest are the same as before.

In [ ]:
def Update(config):
    rate = config.negative / config.positive
    pos = min(config.positive * 2, .1)
    config.positive, config.negative = pos, pos * rate

In [ ]:
device = spyker.device('cuda' if spyker.cuda_available() else 'cpu')

In [ ]:
transform = Transform(device)
network = Network(device)

In [ ]:
from tqdm.notebook import tqdm

for i, (data, _) in enumerate(tqdm(train, "Training Layer 1")):
    if (i + 1) % 10 == 0: Update(network.conv1.stdpconfig[0])
    network.train1(transform(data))
spyker.quantize(network.conv1.kernel, 0, .5, 1)

for i, (data, _) in enumerate(tqdm(train, "Training Layer 2")):
    if (i + 1) % 10 == 0: Update(network.conv2.stdpconfig[0])
    network.train2(transform(data))
spyker.quantize(network.conv2.kernel, 0, .5, 1);

In [ ]:
train_data, train_target = Total(network, transform, train)
test_data, test_target = Total(network, transform, test)

In [ ]:
pca = PCA(n_components=200).fit(train_data, train_target)
train_data, test_data = pca.transform(train_data), pca.transform(test_data)

In [ ]:
target = SVC(C=2.4).fit(train_data, train_target).predict(test_data)
accuracy = (target == test_target).sum() / len(test_target)
print(f"Final Accuracy: {accuracy * 100 :.2f}%")

This implementation might run faster than in the previous tutorial. This might be because we access the datasets directly instead of having TorchVision's MNIST dataset convert them to PIL images and back.